## Verifizierung und Mining

Das dezentrale Fortschreiben der Datenbank basiert auf einigen Regeln - auch Konsensregeln genannt. Dadurch wird sichergestellt, dass der Inhalt der Datenbank (die Besitzverhältnisse der FC) nur verändert werden kann, wenn das entsprechende Regelwerk eingehalten wird. Durch die Integration der asymmetrischen Kryptographie in das Regelwerk wird im Wesentlichen der Währungscharakter erzeugt. Prinzipiell könnte eine Blockchain aber auch ohne Kryptographie implementiert werden. Eine Blockchain ohne Konsensregeln funktioniert jedoch nicht.  
Einen alleinigen Schiedrsichter für die Einhaltung der Regelen gibt es nicht. Die Regelen können verändert werden, sofern die Merheit der Miner diese Veränderung implementiert. *Richtig* bzw. *valide* ist schlichtweg das, was sich in der Blockchain (langfristig) durchsetzt.



### Verifizierung von Transaktionen

Die Validierungsfunktionen werden im Rahmen unsere Implementierung jeweils von der entsprechenden Klasse selbst angeboten. D.h. Transaktionen werden von Methoden innerhalb der `Transaction` Klasse validiert, Blöcke werden von Methoden innerhalb der `Block` Klasse validiert. Da der Austausch von Transaktionen und Blöcken die einzige Kommunikationsfunktion innerhalb unseres Netzwerks darstellt, d.h. hierduch kann die Blockchain verändert werden, beschränkt sich die Validierung auch auf diese beiden Klassen.

#### Implementierung

Damit eine Transaktion *valide* ist, müssen folgenden Kriterien eingehalten werden:
1. Die Signatur muss zum Sender und zur Transaktion passen (`verify_transaction_signature(self)`).
<br/>
<br/>
2. Der Hash der Transaktion muss korrekt sein (`verify_transaction_id()`).
<br/>
<br/>
3. Alle Inputs müssen im Besitz des Senders sein (`verify_inputs(self, blockchain, block_addition = False)`).  
*Zusätzlicher Hinweis: Durch den übergebenen Wert `block_addition` wird der Funktion signalisiert, ob es sich um eine Prüfung im Rahmen einer möglichen Hinzufügung eines Blocks handelt (`=True`) oder einer bestimmten Prüfung im Rahmen einer möglichen Aufnahme einer Transaktion in den Mempool handelt (`=False`), das ist der Standard. Letzteres bedeutet, dass auch überprüft werden soll, ob der verwendete Input bereits von einer konkurrierenden Transaktion im Mempool verwendet wird. Dadurch spart sich er Miner unnötige Arbeit, die beispielsweise entsteht, wenn er eine 'falsche' Transaktion in einen Block steckt, den er gerade minen möchte. Ein solcher Block würde vom Netzwerk nicht akzeptiert werden.*
<br/>
<br/>
4. Summe der Inputs muss gleich Summer der Outputs sein (`verify_sum(self, blockchain)`).  

Der folgende Code implementiert genau diese Funktionen und ist der aktuellsten Version der `Transaction` Klasse, die im folgenden importiert wird, bereits beigefügt. Lediglich die Tests sind noch zu implementieren.




```python

def verify_inputs(self, blockchain, block_addition = False):
    # Check if referenced inputs are UTXOs and if the sender is owner
    # Note that the blockchain object is required for this check.
    
    for input in self.inputs:
        # check if referenced inputs is UTXO
        if input not in blockchain.UTXOs.keys():
            return False
        # Check if referenced input is used in another transaction 
        # which is yet part of the mempool. 
        # In case a new block is added, this is not necessary 
        # (this just for the creation of new transactions, 
        # so that less get rejected).
        if not block_addition:
            if input in blockchain.mempool_UTXOs.keys():
                return False
        # check if referenced inputs is owned by sender of transaction
        parent_block = blockchain.chain[blockchain.UTXOs[input]]
       
        if parent_block.get_output_by_id(input).recipient != self.sender:
            return False
           
    # If all checks went fine, return True     
    return True
    
def verify_sum(self, blockchain):
    # Check if the value of the inputs matches the value 
    # the outputs.

    # Get the value of the inputs
    sum_inputs = 0.0
    for input in self.inputs:
        sum_inputs += float(blockchain.get_UTXO_by_id(input).val)
    # Get the value of the outputs
    sum_outputs = 0.0
    for outputs in self.outputs.values():
        sum_outputs += float(outputs.val
    # Check if input value equals output value
    if sum_inputs == sum_outputs:
        return True
    else:
        return False

def verify_transaction(self, blockchain, block_addition = False):
    # Run through all checks on the Transaction level
    
    if self.verify_transaction_signature() and \
        self.verify_transaction_id() and \
        self.verify_inputs(blockchain, block_addition) and \
        self.verify_sum(blockchain):
        return True
    return False

```


In [7]:
# Automatically relaoad modules that have changed while
# being loaded.
%load_ext autoreload
# Set equal to 2 to enable auto realod,
# set equal to 0 to disable auto relaod.
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Import the outsourced code from the 
# previous section.
from FEDCoin import *
from utils import *

# Create wallets
wallet_genesis = Wallet('308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b020101042012bbcc17335fa7345f7be65b01a5d6dca706d8f2ba99691f314697f288d678c0a144034200043f3bd6d16ce4bde95a8237170aaa106388485498234a3dca5d0c273d907c03c3e72017bec13cf6e893e96da0f9d6c7037c79b40cb006aff12ae88adae1b0bb7e')
wallet_alice = Wallet()

# Create a blockchain object
blockchain = Blockchain()

# Info: Genesis Block_id = '0000002dc75b3bc7ad4b5bce7d572f4d13982e8eb7a67dca6f50d9abeb7f6e81'
# Get the Genesis UTXO_id first:
print(blockchain.chain['0000002dc75b3bc7ad4b5bce7d572f4d13982e8eb7a67dca6f50d9abeb7f6e81'].output_transaction_mapping)

OrderedDict([('13f7b983fb68e4fde6b1b77df955098a2526fe62c965ad435fe61538832bcdb8', 'c4e819034285642fd528308f4b71c2d4f059e58ccf4fea18cc99ec2eb19eb3cb')])


In [9]:
# It's your turn! Try to test the two new verification methods
# in the Transaction class. To this end, create a valid Transaction
# (possibly with change) from the genesis wallet to alice
# and use the blockchain object from above to test the two methods.
# Additional Note: Running the code from above outputs the UTXO from 
# the genesis transaction which belongs to the genesis wallet. Since this is
# the only UTXO available in our system at the moment, you have to use that
# UTXO for your transaction. Also, the code above creates a genesis wallet
# (using the genesis private key) which you have to use to properly create
# the transaction.

# 1. Create TOs: One to alice, one with the change back to genesis

# 2. Create and OD of the TOs

# 3. Create the Transaction from genesis to alice with the OD 
# produced above and the genesis UTXO_id as input.

# 4. Sign the Transaction with the genesis private key

# 5. Check if verify_inputs works 
# (change the input_id to get a 'False' result)
# Note: Use verify_inputs(blockchain, block_addition = False) here.

# 6. Check if verify_sum works
# (change the values of the TOs to get a 'False' result)

# 7. Check if verify_transaction works
# (change the values of either input to get a False result)


In [10]:
# Solution:

# 1. Create TOs: One to alice, one with the change back to genesis
transaction_output_to_alice = TransactionOutput(wallet_alice.public_key, 10)
transaction_output_to_genesis = TransactionOutput(wallet_genesis.public_key, 40)

# 2. Create and OD of the TOs
outputs_od = OrderedDict({transaction_output_to_alice.id:transaction_output_to_alice,
                        transaction_output_to_genesis.id:transaction_output_to_genesis})

# 3. Create the Transaction from genesis to alice with the OD 
# produced above and the genesis UTXO_id as input.
transaction_genesis_to_alice = Transaction(wallet_genesis.public_key,
                                       ['13f7b983fb68e4fde6b1b77df955098a2526fe62c965ad435fe61538832bcdb8'],
                                       outputs_od)

# 4. Sign the Transaction with the genesis private key
transaction_genesis_to_alice.sign_transaction(wallet_genesis.private_key)

# 5. Check if verify_inputs works 
# (change the input_id to get a 'False' result)
print('Verify inputs:')
print(transaction_genesis_to_alice.verify_inputs(blockchain))

# 6. Check if verify_sum works
# (change the values of the TOs to get a 'False' result)
print('Verify sum:')
print(transaction_genesis_to_alice.verify_sum(blockchain))


# 7. Check if verify_transaction works
# (change the values of either input to get a False result)
print('Verify Transaction:')
print(transaction_genesis_to_alice.verify_transaction(blockchain))


Verify inputs:
True
Verify sum:
True
Verify Transaction:
True


### Hinzufügen von Transaktionen zum Mempool der Blockchain

Wird eine Transaktion von einer Instanz der `Blockchain` Klasse als valide eingestuft, so wird sie dem *Mempool* dieser Instanz hinzugefügt. Die Inputs einer validen Transaktion werden außerdem in das `mempool_UTXOs` *OrderedDict* geschrieben. Dadurch kann effizient geprüft werden, welche UTXOs von Transaktionen in *Mempool* geblockt sind. Die folgende Methode `process_transaction` der `Blockchain` Klasse realisiert das Prüfen und gegebenenfalls Hinzufügen einer Transaktion zum *Mempool*. 

```python

def process_transaction(self, transaction):
    # Checks validity of TA, adds it to Mempool
    # and adds inputs of TA to mempool_UTXOs

    # Check if transaction is valid given the current state of the chain
    if transaction.verify_transaction(self):
        # Add transaction to the mempool
        self.mempool[transaction.id] = transaction
        # Add UTXOs of the transaction to the OD of used UTXOs in the mempool
        for input in transaction.inputs:
            self.mempool_UTXOs[input] = transaction.id
        return(True)
    else:
        return(False)    

```

Die Methode ist in der aktuellen Version der `Blockchain` Klasse, die in diesem Verzeichnis liegt, schon beigefügt.

In [11]:
## Testing it by using the transaction created above:

blockchain.process_transaction(transaction_genesis_to_alice)

print(blockchain.mempool)
print("\n")
print(blockchain.mempool_UTXOs)

OrderedDict([('bba4979d8cdbb609107cc5425774d12332b93459d53892b20912cb3563bb7b52', <FEDCoin.Transaction object at 0x10a394978>)])


OrderedDict([('13f7b983fb68e4fde6b1b77df955098a2526fe62c965ad435fe61538832bcdb8', 'bba4979d8cdbb609107cc5425774d12332b93459d53892b20912cb3563bb7b52')])


### Verifizierung von Blöcken

Die Konsensregeln bzgl. der Transaktionen sind eine Teilmenge der Konsensregeln für einen Block. Damit ein Block gültig ist, müssen alle darin enthaltenen Transaktionen gültig sein und zusätzlich muss eine Lösung des POW Rätsels geliefert weren. Außerdem müssen noch einige formale Kriterien geprüft werden (max. Anzahl der Transaktionen pro Block nicht überschritten und Anzahl der Transaktionen größer 0, Hash des Blocks korekt übermittelt).

#### Implementierung

1. Weil zusätzlich zu den herkömmlichen Transaktionen in jedem Block vom Miner noch die *Coinbase Transaktion* hinzugefügt wird und diese eine andere Struktur als die klassischen Transaktionen hat, soll die Coinbase Transaktion von der Blockmethode separat überprüft werden (`verify_coinbase_transaction(self, blockchain)`). Die Coinbase Transaktion enthält die neue geschürften FCs. Da diese Coins zum Zeitpunkt des Schürfens niemand gehören, muss die Transaktion auch nicht signiert werden. Im Senderfeld soll efinfach 'coinbase' stehen.
<br/>
<br/>
2. In `verify_block(self, blockchain)` wird überpürft,
    1. ob alle im Block enthaltenen Transaktionen gültig sind (beginnend mit der *Coinbase Transaktion* durch Aufruf der Methode aus 1.),
    2. ob die max. Anzahl der zulässigen Transaktionen pro Block (in der Blockchainklasse als Klassenvariable hinterlegt) nicht überschritten wird, 
    3. ob die übermittelte Block_id stimmt und 
    4. ob die Lösung des POW Rätsels valide ist.  

Der folgende Code implementiert diese beiden Methoden und ist der aktuellsten Version der `Block` Klasse, die in diesem Notebook importiert wird, bereits beigefügt. Es ist sinnvoll zuerst das Mining zum implementieren, bevor der Code getestet wird.

```python

def verify_coinbase_transaction(self, blockchain):
    # Verify the correctness of the coinbase transaction.
    
    # The coinbase transaction is always the first transaction in a block
    coinbase_transaction = next(iter(self.transactions.values()))
    
    # Check if the sender is equal to 'coinbase'
    if coinbase_transaction.sender != 'coinbase':
        return False
    
    # '//'  performs a floor division, i.e. 5 // 2 = 2, whereas 5 / 2 = 2.5
    # ** is the power operator, i.e. 5**2 = 25
    # Check if the reward is correct
    parent_height = len(blockchain.chain)-1
    correct_reward = blockchain.initial_block_reward * (1/2)**((parent_height + 1) // 
        blockchain.half_time_in_blocks)
    coinbase_output = next(iter(coinbase_transaction.outputs.values()))
    if coinbase_output.value != correct_reward:
        return False
        
    # Check if the there is only one output
    if len(coinbase_transaction.outputs) != 1:
        return False
        
    # Check if the id of the coinbase transaction is right
    if  not coinbase_transaction.verify_transaction_id():
        return False
    return True
    

def verify_block(self, blockchain):
    # Check the correctness of the entire block
    
    # Verify coinbase transaction
    if not self.verify_coinbase_transaction(blockchain):
        return False

    # Verify the other transactions
    trans_counter = 0
    for transaction in self.transactions.values():
        # Skip the first transaction since this is 
        # the coinbase transaction.
        if trans_counter > 0:
            if not transaction.verify_transaction(blockchain, True):
                return False
        trans_counter += 1
    # At least one transaction in addition to the coinbase 
    # transaction must be part of each block
    if trans_counter == 0:
        return False
        
    # If there are more transactions than allowed, reject the block
    if trans_counter > \
        blockchain.max_block_size_in_transactions + 1:
        return False

    # Verify the hash of the block
    if self.id != hash_stuff(self.odict_block()):
        return False

    # Verify difficulty
    if self.id[:blockchain.initial_difficulty] != \
        '0'*blockchain.initial_difficulty:
        return False
    return True

```

### Mining

Die Bündelung von Transaktionen aus dem *Mempool* zu einem Block und die Lösung des damit verbundenen numerischen Rätsels wird als Mining bezeichnet. Letztlich ist beim Mining nur das Resultat relevant. Das Resultat sollte ein Block sein, der von der Mehrheit der Netzwerkteilnehmer als valide eingestuft wird. Wie genau der Miner zu diesem Resultat gelangt, bleibt ihm selbst überlassen. 

#### Implementierung
Das Mining wird in der `Block` Klasse implementiert. Ein sinnvolles Vorgehen - als Prozess aufgeteilt in drei Schritte - könnte z.B. wie folgt aussehen:


1. **`create_coinbase_transaction(self, public_key_receiver)`:**  
   Die Methode erzeugt die *Coinbase* Transaktion, die zum aktuellen Zustand der Blockchain passt. Hierfür muss der richtige *Coinbase* Betrag (Anzahl der neu zu schürfenden Coins in diesem Block) in Abhängigkeit der Parameter `initial_block_reward` und `half_time_in_blocks` ermittelt werden. Der `half_time_in_blocks` gibt an nach welcher Anzahl von Blöcken der `initial_block_reward` jeweils halbiert werden muss. 
   Bei der Generierung des *Coinbase* `TransaktionOutputs` soll `random=0` und `timestamp=0` übergeben werden (das ist willkürlich so festgelegt und erleichtert uns im Folgenden die Identifikation eines *Coinbase* `TransaktionOutputs`). Der *Coinbase* `TransaktionOutputs` wird dann bei der Erzeugung der *Coinbase* `Transaction` verwendet. Bei der Generierung der *Coinbase* `Transaction` muss `sender_pub_key = 'coinbase'` und `inputs = ['coinbase_UTXO']` (auch das ist willkürlich so festgelegt), als output ein `OrderedDict` mit `Coinbase_TransactionOutput.id:Coinbase_TransactionOutput` und `timestamp = 0` (auch Willkür) übergeben werden.
   <br/>
   <br/>
2. **`mine_block(self, public_key_receiver_coinbase_transaction)`:**
   1. Beginne zu minen sobald mindestens eine Transaktion im Mempool ist
   2. Erzeuge die zum aktuellen Zustand der Blockchain passende *Coinbase* Transaktion als erste Transaktion eines OrderedDicts, das im Foldenden als Container für alle Transaktionen, die in den Block gehen sollen, dient. Ziel dieser *Coinbase* Coins sollte das eigene Wallet sein (d.h. eine Wallet auf die der Miner zugriff hat). *Hinweis: Hierfür muss der richtige Coinbase Betrag (Anzahl der neu zu schürfenden Coins in diesem Block) in Abhängigkeit der Parameter `initial_block_reward` und `half_time_in_blocks` ermittelt werden. Der `half_time_in_blocks` gibt an nach welcher Anzahl von Blöcken der `initial_block_reward` jeweils halbiert werden muss.*  
   3. Füge solange Transaktionen zum OrderedDict aus 2. hinzu, bis die maximale Blockgröße erreicht wurde (definiert durch `max_block_size_in_transactions`). Die *Coinbase* Transaktion zählt nicht zu den Transaktionen die bzgl. der. `max_block_size_in_transactions` relevant sind.  
   4. Erzeuge das Block-Objekt mit dem OrderedDict aus 2. und 3. und dem Hashwert des vorangegagenen Blocks in der Blockchain.
   5. Generiere mit der `Block.odict_block()` Methode ein OrderedDict des gesamten Blocks.
   6. Berechne den Hash dieses OrderedDicts (das ist der Hash des Blocks) und verändere das `nonce` Feld solange bis der Hash die in `initial_difficulty` geforderte Anzahl an Nullen zu Beginn hat  (Brute Force).  
   7. Sobald dieses numerische Rätsel gelöst ist, gib das Blockobjekt mit entsprechendem `nonce` Wert zurück.
   <br/>
   <br/>
3. **`add_block()`:**<br/>
    Füge den Block in in die Blockchain ein.

Der zugehörige Code kann folgendermaßen aussehen:

```python

def create_coinbase_transaction(self, public_key_receiver):
    # Creates the coinbase transaction for the next block
    
    # Since we start at zero for measuring the height
    parent_height = len(self.chain)-1 
    
    # '//'  performs a floor division, i.e. 5 // 2 = 2, whereas 5 / 2 = 2.5
    # ** is the power operator, i.e. 5**2 = 25
    coinbase_reward = self.initial_block_reward * (1/2)**((parent_height + 1) \
        // self.half_time_in_blocks)
    coinbase_output= TransactionOutput(public_key_receiver, coinbase_reward, \
        0, timestamp = 0)
    coinbase_transaction = Transaction('coinbase', ['coinbase_UTXO'], \
        OrderedDict({coinbase_output.id:coinbase_output}),0)
    return(coinbase_transaction)
  

def mine_block(self, public_key_receiver_coinbase_transaction):
    # Creates a valid block from the transactions in the mempool.
    # public_key_receiver_coinbase_transaction should be a public key belonging 
    # to the miner of the block.

    # Mine only if the mempool is not empty
    if self.mempool:

        # Block construction
        transactions_odict = OrderedDict()

        # Create the Coinbase transaction
        coinbase_transaction = \
        self.create_coinbase_transaction(public_key_receiver_coinbase_transaction)
        
        transactions_odict[coinbase_transaction.id] = coinbase_transaction

        normal_transaction_counter = 0
        for transaction_id, transaction in self.mempool.items():
            # If max allowed # of transactions are collected
            if normal_transaction_counter >= self.max_block_size_in_transactions:
                break # Get out of the for-loop
            transactions_odict[transaction_id] = transaction
            normal_transaction_counter += 1

        candidate_block = Block(transactions_odict,
                                list(self.chain.values())[len(self.chain)-1].id) 
                                # prev hash
                                
        # Solve Proof of work
        candidate_block_odict = candidate_block.odict_block()
        candidate_block_id = hash_stuff(candidate_block_odict)

        # Check if the first #initial_difficulty figures of the hash are equal to 0
        # if not  continue mining
        # Note: Mining is done on the OrderedDict version
        while not candidate_block_id[:self.initial_difficulty] == \
        '0'*self.initial_difficulty:
            candidate_block_odict['nonce'] = candidate_block_odict['nonce']+1
            candidate_block_id = hash_stuff(candidate_block_odict)

        # If mining was successful, put the solution into the Block() object
        candidate_block.nonce = candidate_block_odict['nonce']
        candidate_block.id = candidate_block_id

        # Add block to the chain
        return(candidate_block)

    # If there is no transaction in the mempool no block can be mined
    return(None)

```




In [12]:
# Testing Block validation and mining:
# Note that the previous cells have to be executed before this test.
# Otherwise the mempool is empty and new_block will be equal to 'None'.
# Also the code cannot be (successfully) run twice in a row, since mempool is empty
# after the first run.

wallet_marcel = Wallet()
print(wallet_marcel.private_key)

# Mine the block
new_block = blockchain.mine_block(wallet_marcel.public_key)

# Add it to the blockchain
blockchain.add_block(new_block)

# Check the the chain length
print(len(blockchain.chain))
print('\n')

# Check pow 
print(blockchain.pow)
print('\n')

# Check mempool (should be empty)
print(blockchain.mempool)
print('\n')

# Check UTXOs (should contain 3 entries, all belonging to the most recent block)
print(blockchain.UTXOs)
print('\n')
print(new_block.id)


308187020100301306072a8648ce3d020106082a8648ce3d030107046d306b0201010420f185a09947de39589a0b324abb9d91115a6fd6b969564013e263a90b7304bda9a1440342000499927dda025b644f8e39bebdcdd0fe52c4264e5ddaaac25db23901bf380fff6cc2ec8181b870691487dfffd886c4e13e2f6f50561cbbda64edf9e71fac8e5879
2


8


OrderedDict()


OrderedDict([('b364ae4c7b360d46781a0d21b0b7970a5d777f2720a87822d4a1a84e61e5fca0', '00000ee3808ee3f0b164958b2cf60d65b8375bd580812a50c938ccb7033f4d69'), ('751923c9cec63297c52412a98aeebc1f8d45a47df57ebdde68ea9c2de794323a', '00000ee3808ee3f0b164958b2cf60d65b8375bd580812a50c938ccb7033f4d69'), ('d7cbc9e9f6935deb874b48bbde79f4838f1fa3741a1d2a80e1e972526d2d3bb0', '00000ee3808ee3f0b164958b2cf60d65b8375bd580812a50c938ccb7033f4d69')])


00000ee3808ee3f0b164958b2cf60d65b8375bd580812a50c938ccb7033f4d69
